In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [2]:
# Load the MNIST dataset
digits = load_digits()
X = digits.images
y = digits.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data
X_train = X_train.reshape(-1, 8, 8, 1)
X_test = X_test.reshape(-1, 8, 8, 1)

# Normalize the input data
X_train = X_train / 16.0
X_test = X_test / 16.0

# Define the CNN architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(8, 8, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))



In [3]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
45/45 [==============================] - 1s 10ms/step - loss: 2.2353 - accuracy: 0.3152 - val_loss: 2.0962 - val_accuracy: 0.5306
Epoch 2/10
45/45 [==============================] - 0s 4ms/step - loss: 1.9133 - accuracy: 0.6903 - val_loss: 1.6882 - val_accuracy: 0.7750
Epoch 3/10
45/45 [==============================] - 0s 4ms/step - loss: 1.4370 - accuracy: 0.8205 - val_loss: 1.1523 - val_accuracy: 0.8528
Epoch 4/10
45/45 [==============================] - 0s 4ms/step - loss: 0.9614 - accuracy: 0.8803 - val_loss: 0.7784 - val_accuracy: 0.8806
Epoch 5/10
45/45 [==============================] - 0s 4ms/step - loss: 0.6618 - accuracy: 0.9012 - val_loss: 0.5521 - val_accuracy: 0.8917
Epoch 6/10
45/45 [==============================] - 0s 3ms/step - loss: 0.4989 - accuracy: 0.9186 - val_loss: 0.4482 - val_accuracy: 0.9111
Epoch 7/10
45/45 [==============================] - 0s 4ms/step - loss: 0.4054 - accuracy: 0.9235 - val_loss: 0.3645 - val_accuracy: 0.9083
Epoch 8/10
45/45 [=

In [4]:
# Evaluate the model
accuracy = model.evaluate(X_test, y_test)[1]
print("Accuracy:", accuracy)

12/12 [==============================] - 0s 2ms/step - loss: 0.2523 - accuracy: 0.9361
Accuracy: 0.9361110925674438
